<a href="https://colab.research.google.com/github/erikrozi/acmlab/blob/main/unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the standard ML libraries...
%load_ext autoreload
%autoreload 2

import pandas as pd                     # to process our data
import matplotlib.pyplot as plt         # graphing
import numpy as np                      # matrices

import torch
import torchvision                      # for MNIST dataset/working with images

# take advantage of GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class TileImageryDataset(Dataset):
  """Tile imagery dataset."""

  def __init__(self, images_zip_path):
    # Load into tensors

    # Perform data augmentation

  def load_images(images_zip_path):
    for x in range(2794, 2839):
      for y in range(6528, 6572):
        

SyntaxError: ignored